In [8]:
#step 1 preprocessing data

# Import library
import pandas as pd

# Membaca dataset dari file .tsv
raw_data = pd.read_csv(r'E:\Bebeb\NEW\AES\data\asap-aes\training_set_rel3.tsv', sep='\t', encoding='ISO-8859-1')

# Menyaring data untuk hanya mengambil essay_set dengan nilai 7
filtered_dataset = raw_data[raw_data['essay_set'] == 7].copy()

# Menyimpan dataset yang telah disaring ke dalam file CSV
filtered_dataset.to_csv('aes_dataset_value_7.csv', index=False)

# Menentukan kolom yang akan digabungkan untuk skor Struktur
skor_struktur = ['rater1_trait1', 'rater1_trait2', 'rater1_trait3', 'rater2_trait1', 'rater2_trait2', 'rater2_trait3']

# Menghitung skor struktur sebagai rata-rata dari 6 karakteristik yang ada
filtered_dataset.loc[:, 'skor_struktur'] = filtered_dataset[skor_struktur].sum(axis=1) / len(skor_struktur)

# Menentukan kolom yang akan digabungkan untuk skor Tata Bahasa
skor_tata_bahasa = ['rater1_trait4', 'rater2_trait4']

# Menghitung skor tata bahasa sebagai rata-rata dari 2 karakteristik yang ada
filtered_dataset.loc[:, 'skor_tata_bahasa'] = filtered_dataset[skor_tata_bahasa].sum(axis=1) / len(skor_tata_bahasa)

# Normalisasi skor Struktur dalam rentang 0-10
min_value_struktur = filtered_dataset['skor_struktur'].min()
max_value_struktur = filtered_dataset['skor_struktur'].max()
filtered_dataset['skor_struktur_normalized'] = 10 * (filtered_dataset['skor_struktur'] - min_value_struktur) / (max_value_struktur - min_value_struktur)
filtered_dataset['skor_struktur_normalized'] = filtered_dataset['skor_struktur_normalized'].round(1)

# Normalisasi skor Tata Bahasa dalam rentang 0-10
min_value_tata_bahasa = filtered_dataset['skor_tata_bahasa'].min()
max_value_tata_bahasa = filtered_dataset['skor_tata_bahasa'].max()
filtered_dataset['skor_tata_bahasa_normalized'] = 10 * (filtered_dataset['skor_tata_bahasa'] - min_value_tata_bahasa) / (max_value_tata_bahasa - min_value_tata_bahasa)
filtered_dataset['skor_tata_bahasa_normalized'] = filtered_dataset['skor_tata_bahasa_normalized'].round(1)

# Pilih kolom-kolom yang relevan untuk disimpan dalam dataset hasil pre-processing
pre_processing_data = ['essay_id', 'essay_set', 'essay', 'skor_struktur_normalized', 'skor_tata_bahasa_normalized']

# Membuat dataset akhir yang berisi data yang sudah diproses
final_pre_processing_data = filtered_dataset[pre_processing_data]

# Menyimpan data hasil pre-processing ke dalam file CSV
final_pre_processing_data.to_csv('pre_processing_data.csv', index=False)

print("Proses pre-processing selesai, file telah disimpan sebagai 'pre_processing_data.csv'")


Proses pre-processing selesai, file telah disimpan sebagai 'pre_processing_data.csv'


In [2]:
import torch

# Test if CUDA is available and the model is using the GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"CUDA is available. Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device('cpu')
    print("CUDA is not available. Using CPU.")

# Example tensor operation on GPU
tensor = torch.rand(3, 3).to(device)
print(tensor)


CUDA is available. Using GPU: NVIDIA GeForce RTX 2060
tensor([[0.7556, 0.1848, 0.9621],
        [0.0797, 0.6222, 0.9174],
        [0.1389, 0.4282, 0.6521]], device='cuda:0')


In [21]:
torch.cuda.empty_cache()


In [30]:
import torch
import torch.nn as nn
import pandas as pd
from transformers import BertTokenizer, BertModel
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

# Load preprocessed dataset
data_preprocessing = pd.read_csv('pre_processing_data.csv')

# Initialize BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Move BERT model to the device (CUDA or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Extract essay texts
essay_texts = data_preprocessing['essay'].tolist()

# Tokenize essays with padding and truncation
inputs = tokenizer(essay_texts, return_tensors='pt', padding=True, max_length=512, truncation=True)

# Extract token IDs and attention mask
input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']

# Move tokenized inputs to the device
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)

# Define the Structure Scoring Model (LSTM + Dropout + Dense)
class StructureScoreModel(nn.Module):
    def __init__(self, hidden_size=768):  # Input size (BERT hidden size)
        super(StructureScoreModel, self).__init__()
        # LSTM Layer 1 with 400 hidden units (Bidirectional, so output size will be 800)
        self.lstm1 = nn.LSTM(input_size=hidden_size, hidden_size=400, batch_first=True, bidirectional=True)
        # LSTM Layer 2 with 128 hidden units (Bidirectional, so output size will be 256)
        self.lstm2 = nn.LSTM(input_size=800, hidden_size=128, batch_first=True, bidirectional=True)
        # Dropout Layer to prevent overfitting
        self.dropout = nn.Dropout(0.5)
        # Dense Layer with ReLU activation to output a single score
        # The input to the dense layer will have size 256 (output from second LSTM, bidirectional)
        self.dense = nn.Linear(128 * 2, 1)  # Output layer for the score (128 * 2 for bidirectional)
        self.relu = nn.ReLU()

    def forward(self, x):
        # Pass through first LSTM layer
        x, _ = self.lstm1(x)  # Output size will be (batch_size, seq_len, 800) due to bidirectional LSTM
        # Pass through second LSTM layer
        x, _ = self.lstm2(x)  # Output size will be (batch_size, seq_len, 256) due to bidirectional LSTM
        # Apply dropout to the output of the second LSTM
        x = self.dropout(x)
        # Take the output from the last sequence token (seq_length - 1)
        x = x[:, -1, :]  # Select the last token (from sequence length)
        # Apply dense layer and ReLU activation
        x = self.dense(x)
        x = self.relu(x)
        return x

# Initialize the model and move to device
model_structure = StructureScoreModel(hidden_size=768).to(device)

# Prepare structure scores for training
structure_scores = torch.tensor(data_preprocessing['skor_struktur_normalized'].values, dtype=torch.float).to(device)

# Split data into training and validation sets
train_input_ids, val_input_ids, train_attention_mask, val_attention_mask, train_scores, val_scores = train_test_split(
    input_ids.cpu(), attention_mask.cpu(), structure_scores.cpu(), test_size=0.2, random_state=42)

# Create TensorDataset and DataLoader for batching
train_dataset = TensorDataset(train_input_ids, train_attention_mask, train_scores)
val_dataset = TensorDataset(val_input_ids, val_attention_mask, val_scores)
batch_size = 2
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Define loss function and optimizer
criterion = nn.MSELoss()  # Mean Squared Error for regression task
optimizer = torch.optim.Adam(model_structure.parameters(), lr=1e-5)

# Function to discretize continuous scores into categories
def discretize_scores(scores, bins=10):
    min_score = np.min(scores)
    max_score = np.max(scores)
    bin_edges = np.linspace(min_score, max_score, bins + 1)
    discretized_scores = np.digitize(scores, bin_edges) - 1  # Subtract 1 to make categories 0-indexed
    return discretized_scores

# Function to scale the scores to a range of 1-10
def scale_scores_to_1_10(scores):
    min_score = np.min(scores)
    max_score = np.max(scores)
    scaled_scores = 1 + 9 * (scores - min_score) / (max_score - min_score)
    return scaled_scores

# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    model_structure.train()
    optimizer.zero_grad()

    for batch in train_dataloader:
        input_ids_batch, attention_mask_batch, structure_scores_batch = batch
        input_ids_batch = input_ids_batch.to(device)
        attention_mask_batch = attention_mask_batch.to(device)
        structure_scores_batch = structure_scores_batch.to(device)

        with torch.no_grad():
            outputs = model(input_ids_batch, attention_mask=attention_mask_batch)

        bert_embeddings = outputs.last_hidden_state
        predictions = model_structure(bert_embeddings)

        loss = criterion(predictions.squeeze(), structure_scores_batch)
        loss.backward()
        optimizer.step()

    # Print training loss
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

    # Validation loop
    model_structure.eval()
    predicted_structure_scores = []
    true_structure_scores = []
    val_loss = 0.0
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids_batch, attention_mask_batch, structure_scores_batch = batch
            input_ids_batch = input_ids_batch.to(device)
            attention_mask_batch = attention_mask_batch.to(device)
            structure_scores_batch = structure_scores_batch.to(device)

            outputs = model(input_ids_batch, attention_mask=attention_mask_batch)
            bert_embeddings = outputs.last_hidden_state

            predictions = model_structure(bert_embeddings)

            val_loss += criterion(predictions.squeeze(), structure_scores_batch).item()

            predicted_structure_scores.extend(predictions.cpu().numpy().flatten())
            true_structure_scores.extend(structure_scores_batch.cpu().numpy())

    val_loss = val_loss / len(val_dataloader)

    # Scale predictions to range 1-10
    predicted_structure_scores_scaled = scale_scores_to_1_10(predicted_structure_scores)
    true_structure_scores_scaled = scale_scores_to_1_10(true_structure_scores)

    # Calculate QWK
    qwk_score = cohen_kappa_score(discretize_scores(predicted_structure_scores_scaled), discretize_scores(true_structure_scores_scaled), weights='quadratic')

    print(f"Validation Loss: {val_loss:.4f}")
    print(f"Validation QWK Score: {qwk_score:.4f}")

# Save predictions to CSV
predictions_df = pd.DataFrame({
    'true_scores': true_structure_scores_scaled,
    'predicted_scores': predicted_structure_scores_scaled
})
predictions_df.to_csv('predicted_structure_scores_1_10.csv', index=False)

print("Training complete!")


c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\loss.py:608: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/50], Loss: 1.5878
Validation Loss: 5.4113
Validation QWK Score: -0.0037


c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\loss.py:608: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [2/50], Loss: 6.9367
Validation Loss: 4.3349
Validation QWK Score: 0.1422


c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\loss.py:608: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


...

Epoch [48/50], Loss: 0.5760
Validation Loss: 1.9934
Validation QWK Score: 0.7047


c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\loss.py:608: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [49/50], Loss: 1.4777
Validation Loss: 2.2240
Validation QWK Score: 0.7638


c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\loss.py:608: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [50/50], Loss: 0.2541
Validation Loss: 1.6245
Validation QWK Score: 0.7599
Training complete!
